In [104]:
import pandas as pd
import numpy as np
import torch
random_seed = 123
np.random.seed(random_seed)

df_1 = pd.read_csv('normal_vec.csv', header=None)
df_2 = pd.read_csv('abnormal_vec.csv', header=None)
df_3 = pd.read_csv('error_vec.csv', header=None)

In [105]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def normal_make_instance(df):
    scaler = StandardScaler()
    noise = np.random.normal(0, 7, (15,6))
    list_scaled = scaler.fit_transform(df + noise)
    abc = np.concatenate((np.array([[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]),list_scaled,np.array([[0,0,0,0,0,0],[0,0,0,0,0,0]])))
    b = torch.from_numpy(abc)
    instance = [b, 0]
    return instance

def abnormal_make_instance(df):
    scaler = StandardScaler()
    noise = np.random.normal(0, 7, (19,6))
    list_scaled = scaler.fit_transform(df + noise)
    abc = np.concatenate((list_scaled, np.array([[0,0,0,0,0,0]])))
    b = torch.from_numpy(abc) 
    instance = [b, 1]
    return instance

def error_make_instance(df):
    scaler = StandardScaler()
    noise = np.random.normal(0, 7, (20,6))
    list_scaled = scaler.fit_transform(df + noise)
    b = torch.from_numpy(list_scaled)
    instance = [b, 2]
    return instance

In [106]:
def make_ds(df_nor, df_ab, df_er, num):
    dataset = []
    for i in range(num):
        dataset.append(normal_make_instance(df_nor))
        dataset.append(abnormal_make_instance(df_ab))
        dataset.append(error_make_instance(df_er))
        
    return dataset

In [107]:
total_dataset = make_ds(df_1, df_2, df_3, 10000)

In [108]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = random_seed)
indices = list(range(len(total_dataset)))
y_ds = [y for _, y in total_dataset]

for train_index, test_index in sss.split(indices, y_ds):
    print('train:', train_index, 'test:', test_index)
    print(len(train_index), len(test_index))

sub_train = Subset(total_dataset, train_index)
test_ds = Subset(total_dataset, test_index)

sss2 = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = random_seed)
indices2 = list(range(len(sub_train)))
y_ds2 = [y for _, y in sub_train]

for train_index2, val_index in sss2.split(indices2, y_ds2):
    print('train:', train_index2, 'val:', val_index)
    print(len(train_index2), len(val_index))

train_ds = Subset(sub_train, train_index2)
val_ds = Subset(sub_train, val_index)

train: [ 9465  4990 23348 ...  8551  7658 23236] test: [28621 18384 29068 ... 29031 11744 16112]
24000 6000
train: [20242 12026  6805 ... 21690 23550 22921] val: [ 6628  2421  8359 ... 20429 12307 18328]
18000 6000


In [109]:
image, label = train_ds[0]
image.size()

torch.Size([20, 6])

In [110]:
from torch.utils.data import DataLoader
batch_size = 64
train_loader = DataLoader(train_ds, batch_size= batch_size, shuffle=True, num_workers=8, drop_last=True)
val_loader = DataLoader(val_ds, batch_size= batch_size, shuffle=True, num_workers=8, drop_last=True)
test_loader = DataLoader(test_ds, batch_size= batch_size, shuffle=False, num_workers=8, drop_last=True)

In [111]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=3, num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # setup LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # setup output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def forward(self, input, hidden=None):
        # lstm step => then ONLY take the sequence's final timetep to pass into the linear/dense layer
        # Note: lstm_out contains outputs for every step of the sequence we are looping over (for BPTT)
        # but we just need the output of the last step of the sequence, aka lstm_out[-1]
        lstm_out, hidden = self.lstm(input, hidden)
        logits = self.linear(lstm_out[-1])              # equivalent to return_sequences=False from Keras
        #softmax = F.softmax(logits)

        return logits, hidden

In [112]:
import torch.optim as optim

model = LSTM(input_dim = 6, hidden_dim = 3, output_dim = 3, num_layers = 1).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
#lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, min_lr=1e-6, verbose=True)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

In [113]:
import torch

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def Accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [114]:
for epoch in range(num_epochs):
    hidden_state = None
    model.train()
    
    train_losses = AverageMeter()
    train_accs = AverageMeter()
    for idx, (images, target) in enumerate(train_loader):
        #print(images.shape)
        images = images.permute(1,0,2).cuda()
        target = target.cuda()
        images = images.type(torch.cuda.FloatTensor)
        y_pred, hidden_state = model(images, hidden_state)
        #print("y_pred",y_pred)

        stateful = False
        if not stateful:
            hidden_state = None
        else:
            h_0, c_0 = hidden_state
            h_0.detach_(), c_0.detach_()
            hidden_state = (h_0, c_0)

        train_loss = criterion(y_pred, target)
        #print(train_loss)
        
        train_losses.update(train_loss.item(), images.size(0)) 
        train_acc = Accuracy(y_pred, target)
        train_accs.update(train_acc[0].item(), images.size(0))
        
        model.zero_grad()
        train_loss.backward()  # backward pass
        optimizer.step()  # parameter update
    #lr_scheduler.step(train_loss)

    with torch.no_grad():
        model.eval()
        hidden_state = None
        
        val_losses = AverageMeter()
        val_accs = AverageMeter()
        
        for idx, (images, target) in enumerate(val_loader):
            images = images.permute(1,0,2).cuda()
            target = target.cuda()
            images = images.type(torch.cuda.FloatTensor)

            y_pred, hidden_state = model(images, hidden_state)
            
            val_loss = criterion(y_pred, target)
            val_losses.update(val_loss.item(), images.size(0)) 

            val_acc = Accuracy(y_pred, target)
            val_accs.update(val_acc[0].item(), images.size(0))
            
            stateful = True
            if not stateful:
                hidden_state = None
            else:
                h_0, c_0 = hidden_state
                h_0.detach_(), c_0.detach_()
                hidden_state = (h_0, c_0)
    
    print('Epoch : {} , train_loss : {}, train_acc : {}, val_loss : {}, val_acc : {} '.format(epoch, train_losses.avg, train_accs.avg, val_losses.avg, val_accs.avg))

Epoch : 0 , train_loss : 1.1955670487414052, train_acc : 33.68549822064057, val_loss : 1.1867542997483285, val_acc : 34.038978494623656 
Epoch : 1 , train_loss : 1.178147371129209, train_acc : 34.224866548042705, val_loss : 1.168733089200912, val_acc : 34.81182795698925 
Epoch : 2 , train_loss : 1.1589972620757025, train_acc : 35.94306049822064, val_loss : 1.1475175093579035, val_acc : 38.894489247311824 
Epoch : 3 , train_loss : 1.1336849427308051, train_acc : 44.82317615658363, val_loss : 1.1166184147198994, val_acc : 51.276881720430104 
Epoch : 4 , train_loss : 1.0958537011808347, train_acc : 55.64946619217082, val_loss : 1.0704655217868027, val_acc : 58.31653225806452 
Epoch : 5 , train_loss : 1.0394567327567268, train_acc : 60.220195729537366, val_loss : 1.0065578722184705, val_acc : 61.87836021505376 
Epoch : 6 , train_loss : 0.9759421921285446, train_acc : 63.42304270462633, val_loss : 0.9443369648789847, val_acc : 64.53293010752688 
Epoch : 7 , train_loss : 0.9153545880657074, 

Epoch : 60 , train_loss : 0.217965764348193, train_acc : 96.48576512455516, val_loss : 0.7530316085584702, val_acc : 79.56989247311827 
Epoch : 61 , train_loss : 0.20999951740176653, train_acc : 96.53580960854093, val_loss : 0.7606489031866033, val_acc : 79.48588709677419 
Epoch : 62 , train_loss : 0.2018393055399966, train_acc : 96.64145907473309, val_loss : 0.7299118714947854, val_acc : 79.93951612903226 
Epoch : 63 , train_loss : 0.19393002639567725, train_acc : 96.68594306049822, val_loss : 0.718964286869572, val_acc : 80.27553763440861 
Epoch : 64 , train_loss : 0.18605134882748764, train_acc : 96.76379003558719, val_loss : 0.7132396598656973, val_acc : 80.72916666666667 
Epoch : 65 , train_loss : 0.17837276431695423, train_acc : 96.83607651245552, val_loss : 0.7017685258260338, val_acc : 80.49395161290323 
Epoch : 66 , train_loss : 0.17089813992752298, train_acc : 96.89724199288256, val_loss : 0.7102087386833724, val_acc : 80.10752688172043 
Epoch : 67 , train_loss : 0.1639043482